In [7]:
# Package installations to work on WIRE

! pip install mamba
! mamba install gensim openai -y
! mamba install -c anaconda nltk -y
! mamba install -c conda-forge spacy -y
! mamba install -c conda-forge pyldavis -y
! mamba install -c conda-forge pypdf2 -y
! pip install transformers sentence-transformers
! pip install scikit-network
! pip install git+https://github.com/rwalk/gsdmm.git
! pip install gensim

! python -m spacy download en_core_web_sm

/bin/bash: line 1: mamba: command not found
/bin/bash: line 1: mamba: command not found
/bin/bash: line 1: mamba: command not found
/bin/bash: line 1: mamba: command not found
/bin/bash: line 1: mamba: command not found
  Cloning https://github.com/rwalk/gsdmm.git to /tmp/pip-req-build-rwckqsul
  Running command git clone --filter=blob:none --quiet https://github.com/rwalk/gsdmm.git /tmp/pip-req-build-rwckqsul
  Resolved https://github.com/rwalk/gsdmm.git to commit 4ad1b6b6976743681ee4976b4573463d359214ee
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.4/26.4 MB 22.3 MB/s eta 0:00:0000:0100:01
2023-04-18 23:21:05.288197: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-18 23:21:05.786423: W tensorflow/comp

In [1]:
# imports

import csv
import json
import os
import re
from pprint import pprint
import time
import re
import unicodedata
import os
import sys
from contextlib import contextmanager
from copy import deepcopy
import string
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.manifold import SpectralEmbedding
import pickle
import sknetwork as skn
from random import *

import matplotlib.pyplot as plt
import seaborn as sns


import numpy as np
import pandas as pd

import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel
from gensim.utils import simple_preprocess

import nltk
nltk.download(["names", "stopwords", "state_union", "twitter_samples", "movie_reviews", "averaged_perceptron_tagger", "vader_lexicon", "punkt", "wordnet"])
from nltk import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
stop_words = stopwords.words("english")
from nltk.sentiment.vader import SentimentIntensityAnalyzer
ps = nltk.porter.PorterStemmer()

from gsdmm import MovieGroupProcess

from sentence_transformers import SentenceTransformer

import spacy
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])

from itertools import combinations


from collections import Counter
# set seed for reproducibility
# np.random.seed(493)

[nltk_data] Downloading package names to /home/jovyan/nltk_data...
[nltk_data]   Package names is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package state_union to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package state_union is already up-to-date!
[nltk_data] Downloading package twitter_samples to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package movie_reviews to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Packag

## Functions for Text Cleaning and Topic Modeling (Same as before)

In [9]:
def tokeniz(sentences):
    for sentence in sentences:
        yield (gensim.utils.simple_preprocess(str(sentence), deacc=True))


# function to filter out stopwords
def remove_stopwords(texts):
    return [
        [word for word in simple_preprocess(str(doc)) if word not in stop_words]
        for doc in texts
    ]


# function for lemmatization
def lemmatize(texts, allowed_postags=["NOUN", "ADJ", "VERB"]):
    texts_op = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_op.append(
            [token.lemma_ for token in doc if token.pos_ in allowed_postags]
        )
    return texts_op


def top_words(cluster_word_distribution, top_cluster, values):
    words = []
    for cluster in top_cluster:
        sort_dicts = sorted(
            cluster_word_distribution[cluster].items(), key=lambda k: k[1], reverse=True
        )[:values]
        words.append([w[0] for w in sort_dicts])
        # print('Cluster %s : %s'%(cluster,sort_dicts))
        # print('-'*120)
    words1 = deepcopy(words)
    for i in range(len(words)):
        for j in range(len(words[i])):
            for k in range(len(words)):
                if k != i and words[i][j] in words[k]:
                    try:
                        words1[i].remove(words[i][j])
                        words1[k].remove(words[i][j])
                    except:
                        pass
    return words1


def create_topics_dataframe(data_text, mgp, threshold, topic_dict, stem_text):
    result = pd.DataFrame(columns=["text", "topic", "stems"])
    for i, text in enumerate(data_text):
        result.at[i, "text"] = text
        result.at[i, "stems"] = stem_text[i]
        prob = mgp.choose_best_label(stem_text[i])
        if prob[1] >= threshold:
            result.at[i, "topic"] = topic_dict[prob[0]]
        else:
            result.at[i, "topic"] = "Other"
    return result

In [10]:
@contextmanager
def suppress_stdout():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:
            yield
        finally:
            sys.stdout = old_stdout

In [11]:
# Building Bigram & Trigram Models
# bigram = gensim.models.Phrases(processed_data, min_count=5, threshold=100)
# trigram = gensim.models.Phrases(bigram[processed_data], threshold=100)
# bigram_mod = gensim.models.phrases.Phraser(bigram)
# trigram_mod = gensim.models.phrases.Phraser(trigram)


# function to filter out stopwords
def remove_stopwords(texts):
    return [
        [word for word in simple_preprocess(str(doc)) if word not in stop_words]
        for doc in texts
    ]


# function to create bigrams
def create_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]


# function to create trigrams
def create_trigrams(texts):
    [trigram_mod[bigram_mod[doc]] for doc in texts]


# function for lemmatization
def lemmatize(texts, allowed_postags=["NOUN", "ADJ", "VERB"]):
    texts_op = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_op.append(
            [token.lemma_ for token in doc if token.pos_ in allowed_postags]
        )
    return texts_op

## Organizing Transcript-Topics (Same as Before)
Final data saves with transcript, topic keywords, sentiment, and program

In [13]:
def transcript_cleaning(month):
    #import spacy stuff
    sid = SentimentIntensityAnalyzer()
    nlp = spacy.load("en_core_web_sm")
    NER = spacy.load("en_core_web_sm")

    tr_tp_st = [[], [], [], []]

    gl = pd.read_csv("foxGuestList.csv", encoding="windows-1252")
    fox = open("fox_text.txt", "r", encoding="windows-1252")
    fox = fox.readlines()
    for l in range(len(gl)):
        #######################################
        ##Turning transcript into df of lines##
        #######################################
        lines = pd.DataFrame(
            columns=["Date", "Start_Hour", "Program", "Title", "Speaker", "Line"]
        )
        # print(lines)
        transcript = fox[l]
        meta = gl.iloc[l, :]
        dt = meta["Date"]
        if month in str(dt):
            if randint(1, 100) <= 5:
                print(l)
            chunks = transcript.split("|")
            line = ""
            spkr = ""
            for c in chunks:
                if (
                    "Page" not in c[0:10]
                    and "...." not in c
                    and c[0:15] not in meta["Title"]
                ):
                    # print(c)
                    if ":" in c:
                        c = c.split(":")
                        if str.isupper(c[0]):
                            lines.loc[len(lines)] = [
                                meta["Date"],
                                meta["Start_Hour"],
                                meta["Program"],
                                meta["Title"],
                                spkr,
                                line,
                            ]
                            # print(spkr + ':' + line)
                            spkr = c[0]
                            line = c[1]
                    else:
                        line += c
            lines.loc[len(lines)] = [
                meta["Date"],
                meta["Start_Hour"],
                meta["Program"],
                meta["Title"],
                spkr,
                line,
            ]

            #########################
            ##Keyword and Sentiment##
            #########################

            lines['Sentence'] = lines['Line'].apply(lambda x: [str(sent) for sent in nlp(x).sents])
            sentences = lines.explode('Sentence')
            sentences = sentences.dropna(subset = ['Sentence'])
            sentences["VADER"] = sentences["Sentence"].apply(
                lambda x: list(sid.polarity_scores(x).values())[3]
            )
            word_sentiment_list = []
            w_list = []
            raw_text= ' '.join(lines['Line'])
            text1 = NER(raw_text)
            words = []
            for word in text1.ents:
                if word.label_ not in ['TIME','MONEY','PERCENT','DATE','QUANTITY','ORDINAL','CARDINAL']:
                    words.append(word.text)
            words = Counter(words).most_common()
            words = words[0:15]
            for w in words:
                    try:
                        w1 = w[0]
                        wdf = sentences[sentences['Sentence'].astype(str).str.contains(str(w1))]
                        w_list.append(w1)
                        sent_list = list(wdf["VADER"])
                        word_sentiment_list.append(sent_list)
                    except:
                        print('error with the contains line')

            ######################
            ##Storing Each Topic##
            ######################
            for w in range(len(w_list)):
                if len(word_sentiment_list[w]) > 2:
                    tr_tp_st[0].append(l)
                    tr_tp_st[1].append(w_list[w])
                    tr_tp_st[2].append(np.mean(word_sentiment_list[w]))
                    tr_tp_st[3].append(lines["Program"][0])


    gl = pd.read_csv("msnbcGuestList.csv", encoding="windows-1252")
    fox = open("msnbc_text.txt", "r", encoding="windows-1252")
    fox = fox.readlines()
    for l in range(len(gl)):
        #######################################
        ##Turning transcript into df of lines##
        #######################################
        lines = pd.DataFrame(
            columns=["Date", "Start_Hour", "Program", "Title", "Speaker", "Line"]
        )
        # print(lines)
        transcript = fox[l]
        meta = gl.iloc[l, :]
        dt = meta["Date"]
        if month in str(dt):
            if randint(1, 100) <= 5:
                print(l)
            chunks = transcript.split("|")
            line = ""
            spkr = ""
            for c in chunks:
                if (
                    "Page" not in c[0:10]
                    and "...." not in c
                    and c[0:15] not in meta["Title"]
                ):
                    # print(c)
                    if ":" in c:
                        c = c.split(":")
                        if str.isupper(c[0]):
                            lines.loc[len(lines)] = [
                                meta["Date"],
                                meta["Start_Hour"],
                                meta["Program"],
                                meta["Title"],
                                spkr,
                                line,
                            ]
                            # print(spkr + ':' + line)
                            spkr = c[0]
                            line = c[1]
                    else:
                        line += c
            lines.loc[len(lines)] = [
                meta["Date"],
                meta["Start_Hour"],
                meta["Program"],
                meta["Title"],
                spkr,
                line,
            ]

            #########################
            ##Keyword and Sentiment##
            #########################

            lines['Sentence'] = lines['Line'].apply(lambda x: [str(sent) for sent in nlp(x).sents])
            sentences = lines.explode('Sentence')
            sentences = sentences.dropna(subset = ['Sentence'])
            sentences["VADER"] = sentences["Sentence"].apply(
                lambda x: list(sid.polarity_scores(x).values())[3]
            )
            word_sentiment_list = []
            w_list = []
            raw_text= ' '.join(lines['Line'])
            text1 = NER(raw_text)
            words = []
            for word in text1.ents:
                if word.label_ not in ['TIME','MONEY','PERCENT','DATE','QUANTITY','ORDINAL','CARDINAL']:
                    words.append(word.text)
            words = Counter(words).most_common()
            words = words[0:15]
            for w in words:
                    try:
                        w1 = w[0]
                        wdf = sentences[sentences['Sentence'].astype(str).str.contains(str(w1))]
                        w_list.append(w1)
                        sent_list = list(wdf["VADER"])
                        word_sentiment_list.append(sent_list)
                    except:
                        print('error with the contains line')

            ######################
            ##Storing Each Topic##
            ######################
            for w in range(len(w_list)):
                if len(word_sentiment_list[w]) > 2:
                    tr_tp_st[0].append(l)
                    tr_tp_st[1].append(w_list[w])
                    tr_tp_st[2].append(np.mean(word_sentiment_list[w]))
                    tr_tp_st[3].append(lines["Program"][0])

    gl = pd.read_csv("CNNGuestList.csv", encoding="windows-1252")
    fox = open("cnn_text.txt", "r", encoding="windows-1252")
    fox = fox.readlines()
    for l in range(len(gl)):
        #######################################
        ##Turning transcript into df of lines##
        #######################################
        lines = pd.DataFrame(
            columns=["Date", "Start_Hour", "Program", "Title", "Speaker", "Line"]
        )
        # print(lines)
        transcript = fox[l]
        meta = gl.iloc[l, :]
        dt = meta["Date"]
        if month in str(dt):
            if randint(1, 100) <= 5:
                print(l)
            chunks = transcript.split("|")
            line = ""
            spkr = ""
            for c in chunks:
                if (
                    "Page" not in c[0:10]
                    and "...." not in c
                    and c[0:15] not in meta["Title"]
                ):
                    # print(c)
                    if ":" in c:
                        c = c.split(":")
                        if str.isupper(c[0]):
                            lines.loc[len(lines)] = [
                                meta["Date"],
                                meta["Start_Hour"],
                                meta["Program"],
                                meta["Title"],
                                spkr,
                                line,
                            ]
                            # print(spkr + ':' + line)
                            spkr = c[0]
                            line = c[1]
                    else:
                        line += c
            lines.loc[len(lines)] = [
                meta["Date"],
                meta["Start_Hour"],
                meta["Program"],
                meta["Title"],
                spkr,
                line,
            ]

            #########################
            ##Keyword and Sentiment##
            #########################

            lines['Sentence'] = lines['Line'].apply(lambda x: [str(sent) for sent in nlp(x).sents])
            sentences = lines.explode('Sentence')
            sentences = sentences.dropna(subset = ['Sentence'])
            sentences["VADER"] = sentences["Sentence"].apply(
                lambda x: list(sid.polarity_scores(x).values())[3]
            )
            word_sentiment_list = []
            w_list = []
            raw_text= ' '.join(lines['Line'])
            text1 = NER(raw_text)
            words = []
            for word in text1.ents:
                if word.label_ not in ['TIME','MONEY','PERCENT','DATE','QUANTITY','ORDINAL','CARDINAL']:
                    words.append(word.text)
            words = Counter(words).most_common()
            words = words[0:15]
            for w in words:
                    try:
                        w1 = w[0]
                        wdf = sentences[sentences['Sentence'].astype(str).str.contains(str(w1))]
                        w_list.append(w1)
                        sent_list = list(wdf["VADER"])
                        word_sentiment_list.append(sent_list)
                    except:
                        print('error with the contains line')

            ######################
            ##Storing Each Topic##
            ######################
            for w in range(len(w_list)):
                if len(word_sentiment_list[w]) > 2:
                    tr_tp_st[0].append(l)
                    tr_tp_st[1].append(w_list[w])
                    tr_tp_st[2].append(np.mean(word_sentiment_list[w]))
                    tr_tp_st[3].append(lines["Program"][0])

    return tr_tp_st

In [14]:
# Save out processed and topic modeled data
months = ['January','February','March','April','May','June','July','August','September','October','November','December']
for m in months:
    tr_tp_st = transcript_cleaning(m)
    tdf = pd.DataFrame(zip(tr_tp_st[0], tr_tp_st[1], tr_tp_st[2], tr_tp_st[3]), 
                       columns=["Transcript", "Topic", "Sentiment", "Program"],)
    with open('new processed_transcripts_' + m + '2020.pkl', 'wb') as file:
        pickle.dump(tr_tp_st, file)
    tdf.to_csv('new processed_transcripts_' + m + '2020.csv')

36
90
473
501
740
832
869
1612
2194
2195
2327
2582
2702
2755
2840
430
1400
1409
1560
1632
1756
1866
1869
139
600
1505
1599
1698
2065
2225
2368
2759
3226
3327
3680
3683
3689
3704
3708
3714
4261
4299
4625
4681
error with the contains line
5183
5395
5785
5854
5881
5885
5933
6293
6633
6789
6998
7092
7108
7124
7155
error with the contains line
7237
7743
7830
7844
7941
8555
8690
8697
8831
8929
9102
10037
10043
10420
10659
10727
64
347
754
error with the contains line
error with the contains line
987
1379
2441
2505
2554
449
451
456
529
531
661
764
814
950
1385
1852
1855
1856
38
69
170
201
error with the contains line
error with the contains line
error with the contains line
252
598
error with the contains line
1327
1441
1710
2139
2163
2207
2328
3408
3923
3950
4168
4506
4603
4927
5385
5756
5808
5943
6207
6217
6484
6502
error with the contains line
error with the contains line
6757
6769
6889
6931
6939
6954
7001
7069
7189
7322
7456
7460
7476
7479
7621
9814
10444
329
371
807
error with the contai

/tmp/ipykernel_2632/351936673.py:269: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  wdf = sentences[sentences['Sentence'].astype(str).str.contains(str(w1))]


4823
4836
error with the contains line
5062
error with the contains line
5108
5152
5211
5837
6634
6683
error with the contains line
7990
8351
8386
8908
8944
9119
9208
9253
9430
9520
9568
9573
9623
9948
9979
9989
10080
10236
10289
10575
465
673
1249
1772
1807
2548
7
760
810
922
1596
2053
1066
1224
error with the contains line
1971
2283
2345
2569
error with the contains line
2742
2763
2796
3086
error with the contains line
error with the contains line
3793
3915
3940
4188
4379
4526
4528
4816
4846
5153
5523
5747
5823
6068
6079
6104
6414
6491
6515
6602
6834
6913
7064
7145
7405
7483
7617
7664
8093
error with the contains line
8256
8261
8956
9178
9204
9280
9503
9743
10224
209
253
374
406
1350
1414
1415
1714
1854
1868
1918
1945


/tmp/ipykernel_2632/351936673.py:177: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  wdf = sentences[sentences['Sentence'].astype(str).str.contains(str(w1))]


399
error with the contains line
720
1302
1777
2001
2111
2114
error with the contains line
423
444
651
657
876
1080
1164
1253
2003
2770
3360
3366
3551
3798
4056
4415
5559
5564
6204
6305
6483
7321
7698
7702
7865
8352
8488
8506
8807
8866
9082
9136
9194
9323
9401
9416
9680
9946
10217
10737
68
error with the contains line
553
572
628
653
721
883
897
error with the contains line
956
error with the contains line
1173
1486
1500
2003
2137
2198
2629
2819
267
335
767
error with the contains line
1268
1269
1533
1776
1973
2041
2086
2087
93
335
401
564
706
855
1216
1384
1588
2278
2284
2290
2295
2534
2670
2895
3014
3072
3893
3895
4017
4081
error with the contains line
5254
5647
5692
5762
5792
5897
6141
6205
6358
6382
6413
6465
6466
6564
7772
7912
8149
8188
8221
8282
8611
8637
8656
8678
8771
8864
9161
9176
9189
9660
9805
52
55
89
277
561
629
732
804
error with the contains line
963
1119
1144
1347
1351
error with the contains line
1505
1705
1733
2254
2829
189
309
323
770
1030
1511
error with the conta

In [7]:
print(tr_tp_st[0][:5])
print(tr_tp_st[1][:5])
print(tr_tp_st[2][:5])
print(tr_tp_st[3][:5])
print(len(tr_tp_st[1]))
print(tr_tp_st[3][1440])

[21, 21, 21, 21, 21]
[['generation', 'free', 'vote', 'people', 'man', 'feel', 'see'], ['know', 'word', 'impeachment', 'election', 'evidence', 'use'], ['stage', 'mueller', 'grief', 'question', 'country'], ['information', 'president', 'memo', 'medium', 'leak', 'news', 'receive'], ['tonight', 'liberal', 'way', 'angle', 'good']]
[0.1352142857142857, 0.004306896551724135, -0.026888000000000006, -0.18042962962962963, -0.03984999999999999]
['Fox News Network INGRAHAM ANGLE', 'Fox News Network INGRAHAM ANGLE', 'Fox News Network INGRAHAM ANGLE', 'Fox News Network INGRAHAM ANGLE', 'Fox News Network INGRAHAM ANGLE']
6571
MSNBC 11TH HOUR WITH BRIAN WILLIAMS
